### This part is about how to get the prediction from the account and text and testing the running time of the whole process.
#### By: Yue Yang(UIN: 625008110)

Our web app is built on python 3. Because of the difference between python 2.x and python 3.x, if we want to use the model created by the teammate, we must use the same versin python. So, we need check the version at first.

In [5]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/Users/letheyue/anaconda2/envs/yy_py3/bin/python
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:14:23) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)


In [7]:
import twitter
import time

api = twitter.Api(consumer_key=CONSUMER_KEY,
                     consumer_secret=CONSUMER_SECRET,
                     access_token_key=ACCESS_TOKEN_KEY,
                     access_token_secret=ACCESS_TOKEN_SECRET)

### Step 1: test the account model

#### step 1.1: get the account feature

In [22]:
import pandas as pd
import numpy as np

def get_user(user_id=None, screen_name=None):
    json = api.GetUser(user_id=user_id, screen_name=screen_name, include_entities=True, return_json=False)
    json_data = json._json
    
    # feature Count of favorite tweets
    Count_of_favorite_tweets = int(json_data['favourites_count'])
    # feature Friends to follower ratio
    if json_data['followers_count'] == 0:
        Friends_to_follower_ratio = float(100000)
    else:
        Friends_to_follower_ratio = (float(json_data['friends_count']) / json_data['followers_count'])
    # feature Total status count
    Total_status_count = int(json_data['statuses_count'])
    
    # feature Default profile image & Default profile
    # def_p_na','def_p_false','def_p_true'
    if json_data['default_profile_image'] == 'FALSE':
        Def_p_img_false = 1.0
        Def_p_img_true = 0.0
        Def_p_img_na = 0.0
    elif json_data['default_profile_image'] == 'True':
        Def_p_img_false = 0.0
        Def_p_img_true = 1.0
        Def_p_img_na = 0.0
    else:
        Def_p_img_false = 0.0
        Def_p_img_true = 0.0
        Def_p_img_na = 1.0

    if json_data['default_profile'] == 'FALSE':
        Def_p_false = 1.0
        Def_p_true = 0.0
        Def_p_na = 0.0
    elif json_data['default_profile'] == 'True':
        Def_p_false = 0.0
        Def_p_true = 1.0
        Def_p_na = 0.0
    else:
        Def_p_false = 0.0
        Def_p_true = 0.0
        Def_p_na = 1.0
    
    # feature Account ages
    created_at = json_data['created_at']
    Account_age = survival_time(created_at)

    # feature User name and screen_name
    User_name = json_data['name']
    Screen_name = json_data['screen_name']
    User_name_digit, User_name_char = counter(User_name)
    Screen_name_digit, Screen_name_char = counter(Screen_name)

    # feature Length of description and Description text
    description_pre = json_data['description']
    Description_length = len(description_pre)
    
    # feature Average tweet per day
    # Average_tweets_per_day = Total_status_count / float(Account_age)

    feature = pd.DataFrame(index=[0])
    feature['favorite_count'] = Count_of_favorite_tweets
    feature['friends_to_followers'] = Friends_to_follower_ratio
    feature['statuses_count'] = Total_status_count
    feature['def_p_img_na'] = Def_p_img_na
    feature['def_p_img_false'] = Def_p_img_false
    feature['def_p_img_true'] = Def_p_img_true
    feature['def_p_na'] = Def_p_na
    feature['def_p_false'] = Def_p_false
    feature['def_p_true'] = Def_p_true
    feature['age'] = Account_age
    feature['name_letter'] = User_name_char
    feature['name_num'] = User_name_digit
    feature['screen_letter'] = Screen_name_char
    feature['screen_num'] = Screen_name_digit
    feature['des_len'] = Description_length
#     feature['avg_tweet_per_day'] = Average_tweets_per_day
#     feature['des_text'] = Description_tfidf

    return feature

In [9]:
def survival_time(created_at):
    # get the account ages: crawl at time - created at time 
    current_time = time.localtime(time.time())
    current_year = current_time.tm_year
    current_month = current_time.tm_mon
    current_day = current_time.tm_mday

    meta = created_at.split(" ")
    created_month = meta[1]
    if created_month == 'Jan':
        created_month = int(1)
    elif created_month == 'Feb':
        created_month = int(2)
    elif created_month == 'Mar':
        created_month = int(3)
    elif created_month == 'Apr':
        created_month = int(4)
    elif created_month == 'May':
        created_month = int(5)
    elif created_month == 'Jun':
        created_month = int(6)
    elif created_month == 'Jul':
        created_month = int(7)
    elif created_month == 'Aug':
        created_month = int(8)
    elif created_month == 'Sep':
        created_month = int(9)
    elif created_month == 'Oct':
        created_month = int(10)
    elif created_month == 'Nov':
        created_month = int(11)
    elif created_month == 'Dec':
        created_month = int(12)
    created_day = int(meta[2])
    created_year = int(meta[5])

    Account_age = (current_year - created_year) * 365 + (current_month - created_month) * 30 + (current_day - created_day)

    return Account_age

def counter(name):
    # counter of char & counter of digit
    numbers = sum(c.isdigit() for c in name)
    words   = sum(c.isalpha() for c in name)

    return numbers, words

### measure the running time of the whole process of prediction through account model

In [18]:
from sklearn.externals import joblib

def measure_time(user_id=None, screen_name=None):
    # measure the running time of getting the features from the input
    start = time.clock()
    
    with open("classifier/ensemble_user_2.pkl", "rb") as file_handler:
        loaded_pickle = joblib.load(file_handler)
    feature = get_user(screen_name=screen_name)
    np_feature = np.asarray((feature))
    pred_account = loaded_pickle.predict(np_feature.tolist())
    print(pred_account[0])
    
    end = time.clock()
    print('function took %0.5f ms' % ((end-start)*1000.0))

In [23]:
measure_time(screen_name='realDonaldTrump')

0
function took 72.50400 ms


/Users/letheyue/anaconda2/envs/yy_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Step 2: test the text model

#### Step 2.1 : get the latest tweet of the user

In [11]:
timeline = api.GetUserTimeline(screen_name="realDonaldTrump")

In [12]:
timeline

[Status(ID=985858100149309441, ScreenName=realDonaldTrump, Created=Mon Apr 16 12:31:02 +0000 2018, Text='Russia and China are playing the Currency Devaluation game as the U.S. keeps raising interest rates. Not acceptable!'),
 Status(ID=985856662866202624, ScreenName=realDonaldTrump, Created=Mon Apr 16 12:25:19 +0000 2018, Text='Comey drafted the Crooked Hillary exoneration long before he talked to her (lied in Congress to Senator G), then ba… https://t.co/X4GPSNm0KK'),
 Status(ID=985529299905187840, ScreenName=realDonaldTrump, Created=Sun Apr 15 14:44:30 +0000 2018, Text='Just hit 50% in the Rasmussen Poll, much higher than President Obama at same point. With all of the phony stories a… https://t.co/J2tMvPcCtK'),
 Status(ID=985504808646971392, ScreenName=realDonaldTrump, Created=Sun Apr 15 13:07:10 +0000 2018, Text='Slippery James Comey, a man who always ends up badly and out of whack (he is not smart!), will go down as the WORST… https://t.co/YSTiN7oLWq'),
 Status(ID=98550205334575104

In [13]:
list_text = list()
index = 0
for item in timeline:
    index += 1
    data = item._json
    list_text.append(data['text'])
    if index == 1:
        break

In [14]:
print("The latest tweet of the user is:")
list_text

The latest tweet of the user is:


['Russia and China are playing the Currency Devaluation game as the U.S. keeps raising interest rates. Not acceptable!']

#### step2 : pre-process the text

In [17]:
import re
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r"(?P<url>https?://[^\s]+)"
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner_updated(text):
    try:
        mention = re.search(pat1, text).group()
    except:
        mention = ''
    try:
        url = re.search(pat2, text).group("url")
        o = urlparse(url)
        netloc = o.netloc
        path = p.path
    except:
        netloc = ''
        path = ''
    soup = BeautifulSoup(text, 'html.parser')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\u2026", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    lower_case = stripped.lower()
    letters_only = re.sub("[^a-zA-Z]", " ", lower_case)
    letters_only = letters_only + netloc + path + mention
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [18]:
text = tweet_cleaner_updated(list_text[0])

In [19]:
print("The text of pre-processing is:")
text

The text of pre-processing is:


'russia and china are playing the currency devaluation game as the keeps raising interest rates not acceptable'

#### step 3: convert the text to vector features

In [26]:
import pickle
import numpy as np

with open('pos_hmean.p', 'rb') as fp:
    w2v_pos_hmean_01 = pickle.load(fp, encoding='latin1')

In [27]:
def get_w2v_general(tweet, size, vectors):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [28]:
text = get_w2v_general(text, 200, w2v_pos_hmean_01)

In [29]:
print("The vector after converting we get:")
text

The vector after converting we get:


array([[ 0.26959012, -0.30902003,  0.10471017,  0.05447358,  0.59808705,
        -0.24565055,  0.13758163,  0.8677728 ,  0.10629074,  0.29326281,
        -0.12304791,  0.18731175,  0.08726956, -0.13513876,  0.56175141,
        -0.74654535, -0.83927049,  0.25109404,  0.62193509,  0.16840129,
        -0.327807  ,  0.4963034 ,  0.60733668, -0.04265926, -0.21431528,
        -0.29100448,  0.20359909, -0.52525007, -0.0895632 ,  0.11441323,
        -0.08939255, -0.52311338,  0.54198102, -0.6624052 ,  0.05750201,
        -0.34641455, -0.26974008, -0.04486086,  0.32916476,  0.30484454,
         1.3524896 , -0.19088946, -0.04657965,  0.02105158,  0.51551128,
         0.01641426,  0.25352309,  0.13784522, -0.54720956,  0.12815888,
        -0.38172564,  0.73967721,  0.11105266, -0.2913123 ,  0.27487024,
         0.80724067,  0.36642141, -0.14330417,  1.11421222,  0.56861426,
         0.17612828, -0.33602466, -0.74988724, -0.19811932, -0.47795002,
         0.05230433,  0.12115524, -0.00387796,  0.0

####  step 4: get the prediction

In [30]:
from keras.models import load_model
loaded_w2v_model = load_model('w2v_01_best_weights.10-0.9346.hdf5')

/Users/letheyue/anaconda2/envs/yy_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
pred = loaded_w2v_model.predict(text)

In [32]:
pred[0][0]

1.0

### measure the running time of the whole process of prediction through text model

In [33]:
def measure_running_time(user_id=None, screen_name=None):
    # measure the running time of getting the features from the input
    start = time.clock()
    # get the timeline of user
    timeline = api.GetUserTimeline(screen_name=screen_name)
    
    # get the first text
    list_text = list()
    index = 0
    for item in timeline:
        index += 1
        data = item._json
        list_text.append(data['text'])
        if index == 1:
            break
            
    # pre-processing the data
    text = tweet_cleaner_updated(list_text[0])
    
    # load the function
    with open('pos_hmean.p', 'rb') as fp:
        w2v_pos_hmean_01 = pickle.load(fp, encoding='latin1')
    loaded_w2v_model = load_model('w2v_01_best_weights.10-0.9346.hdf5')
    
    # get the prediction value
    text = get_w2v_general(text, 200, w2v_pos_hmean_01)
    pred = loaded_w2v_model.predict(text)
    print(pred[0][0])
    
    end = time.clock()
    print('function took %0.5f ms' % ((end-start)*1000.0))

In [34]:
measure_running_time(screen_name='realDonaldTrump')

1.0
function took 2559.56100 ms
